In [1]:
import sys
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications import resnet50
import pandas as pd
import numpy as np
import os
import cv2

In [2]:
labels = pd.read_csv('Data/train_label.csv')
sample = pd.read_csv('Data/sample_submission.csv')
# num_train_images = labels.shape[0]
num_train_images = 2520  # we choose 3300 images for this assignment. It works for a machine having 8Gb Ram. You can adjust it if your Ram is different. 
split_point = 2240 # split the data into training data [0:3000] and val data [3000:]
print('num_train_images:', num_train_images)
types = sample.columns[1:]
print('Types:', types)
num_classes = len(types)
print('num_classes:', num_classes)

num_train_images: 2520
Types: Index(['distribute', 'ineq', 'integral', 'limit', 'matrix', 'series', 'sqrt'], dtype='object')
num_classes: 7


In [3]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

img_width = 224

def get_image(filename):
    ########################################################################
    # TODO: Your code here...
    ########################################################################
    original = load_img(filename, target_size=(224,224))
    numpy_image = img_to_array(original)
    image_batch = np.expand_dims(numpy_image, axis=0)
    return image_batch[0]


In [4]:
x_train = np.zeros((num_train_images, img_width, img_width, 3), dtype=np.uint8)
y_train = np.zeros((num_train_images, num_classes), dtype=np.uint8)

count = 0

for i in range(num_train_images):
    x_train[i] = get_image('Data/train/%s.png' % labels['id'][i])
    pos_arrays = (types == labels['Type'][i]).nonzero() # recall that types is the array of classes
    pos = pos_arrays[0][0]
    y_train[i][pos] = 1
    count += 1
    if(count % 1000 == 0): print(count, 'images loaded')

1000 images loaded
2000 images loaded


In [5]:
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.applications import vgg16

#getting the features of all three models

#Resnet50 - last layer of features
resnet50_x_train = resnet50.preprocess_input(x_train.copy())
resnet50_model = resnet50.ResNet50(weights='imagenet', include_top=False)
resnet50_features = resnet50_model.predict(resnet50_x_train, batch_size=64, verbose=1)
print(resnet50_features.shape)

#vgg16
vgg16_x_train = vgg16.preprocess_input(x_train.copy())
vgg16_model = vgg16.VGG16(weights='imagenet', include_top=False)
vgg16_features = vgg16_model.predict(vgg16_x_train, batch_size=64, verbose=1)
print(vgg16_features.shape)

40/40 [==============================] - 195s 5s/step
(2520, 7, 7, 2048)
40/40 [==============================] - 483s 12s/step
(2520, 7, 7, 512)


In [6]:
concat = np.concatenate([resnet50_features, vgg16_features],axis=3)
print(concat.shape)

(2520, 7, 7, 2560)


In [7]:
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, Activation, BatchNormalization

inputs = Input(shape = (7, 7, 2560)) # to take 7 x 7 x 2048 images
x = GlobalAveragePooling2D()(inputs) # to convert to 2048 feagures
print(x.shape)
x = Dropout(0.5)(x) # add a dropout layer
x = Dense(500)(x) # add a dense layer, but not adding activation so that we can add batch-norm first
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x) # add a dropout layer
# Softmax layer to the output classes
predictions = Dense(num_classes, activation='softmax')(x) # arg1 is: units = dimensionality of the output space.
print(predictions.shape)

(None, 2560)
(None, 7)


In [8]:
from tensorflow.keras.models import load_model
model = load_model("./models/model_rv_mae.h5")

## do NOT run this cell if you did not collect you own data. This cell processes the pictures for us.

In [75]:

import math
import os

import cv2
import numpy as np


def make_bitmap(img: np.ndarray):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    filter = cv2.getGaussianKernel(31, 11)
    filter = filter * filter.T
    smoothed_im = cv2.filter2D(img, 0, filter)
    _, img = cv2.threshold(smoothed_im, 115, 255, cv2.THRESH_BINARY)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    return img


def resize_img(im: np.ndarray, dim: tuple):
    y, x = im.shape[0], im.shape[1]
    y_out, x_out = dim

    if y_out > y:
        diff = abs(y_out - y)
        padding = math.ceil(diff / 2)
        im = cv2.copyMakeBorder(im, padding, padding, 0, 0, cv2.BORDER_CONSTANT, None, value=[255, 255, 255])

    if x_out > x:
        diff = abs(y_out - y)
        padding = math.ceil(diff / 2)
        im = cv2.copyMakeBorder(im, 0, 0, padding, padding, cv2.BORDER_CONSTANT, None, value=[255, 255, 255])
    
    im = cv2.resize(im, dim, interpolation= cv2.INTER_CUBIC)
    return im


def display(im):
    cv2.imshow("Image", im)
    cv2.waitKey()
    cv2.destroyAllWindows()



def write_from_dir(read_path: str):
    for file in os.listdir(read_path):
        if os.path.isfile(os.path.join(read_path, file)):
            sub_dir = read_path.split("/")[-1]
            new_dir = "./data/" + sub_dir
            old_filepath = os.path.join(read_path, file)
            new_filepath = os.path.join(new_dir, file)
            os.makedirs(new_dir, exist_ok=True)
            img = cv2.imread(old_filepath)
            if img is not None:
                try:
                    img = resize_img(img, (250, 250))
                except:
                    print(old_filepath)
                cv2.imwrite(new_filepath, img)





dirs = [
    "./Data/Test_8",

]
for dir in dirs:
    write_from_dir(dir)



In [ ]:
Test_prediction = []
for files in os.listdir('./Data/Test_e'):

    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_e/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        
        image_batch1 = resnet50.preprocess_input(image_batch.copy())
        image_batch2 = vgg16.preprocess_input(image_batch.copy())
        feature_input1 = resnet50_model.predict(image_batch1)
        feature_input2 = vgg16_model.predict(image_batch2)
        concat2 = np.concatenate([feature_input1,feature_input2],axis=3)
        predictions = model.predict(concat2)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

        #print('Image Name: ',files,' Prediction: ',types[pos])
#Test_prediction
#Test_prediction = np.array(Test_prediction)

In [32]:
#Test_prediction = np.array(Test_prediction)
y_test = pd.read_excel(r'./Data/test_e_label.xlsx',usecols=[1,1])
y_test = y_test.to_numpy()
y_test = np.squeeze(y_test)
# print(y_test)

In [33]:
test_acc = (Test_prediction == y_test).mean()

print('Test accuracy: ', test_acc)

Test accuracy:  0.6


In [27]:
for files in os.listdir('./Data/Test_Cam/'):
    
    if files== 'desktop.ini':
        pass
    else:    
        test_img = get_image(f'./Data/Test_Cam/{files}')
        image_batch = np.expand_dims(test_img, axis=0)
        image_batch = np.copy(image_batch)
        
        image_batch1 = resnet50.preprocess_input(image_batch.copy())
        image_batch2 = vgg16.preprocess_input(image_batch.copy())
        feature_input1 = resnet50_model.predict(image_batch1)
        feature_input2 = vgg16_model.predict(image_batch2)
        concat2 = np.concatenate([feature_input1,feature_input2],axis=3)
        predictions = model.predict(concat2)
        pos = np.argmax(predictions)
        Test_prediction.append(pos)

        print('Image Name: ',files,' Prediction: ',types[pos])


Image Name:  cam dist.png  Prediction:  ineq
Image Name:  cam ineq.png  Prediction:  ineq
Image Name:  cam int.png  Prediction:  integral
Image Name:  cam limit.png  Prediction:  limit
Image Name:  cam mat.png  Prediction:  integral
Image Name:  cam series.png  Prediction:  series
Image Name:  cam sqrt.png  Prediction:  sqrt
